In [38]:
#NOTE: an explanation of the relevant method here is in the README file for this repository

#FINAL METHOD
#change method based on duration_hours and fractional_duration
#change time-bin size and size of the phase mask
#catch AttributeError and TimeDivisionError
#uses minmax normalization for all curves
#downloads curves to a file on desktop; closes figures once completer 

import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv

data = "finaldata.csv"
fields = []
rows = []

with open(data, 'r') as file:
    read = csv.reader(file)
    fields = next(read)
    for row in read:
        rows.append(row)

path = "/Users/anahitasrinivasan/Desktop/UROP Items/Lightcurves/"

for x in range(100, 200):
    try:
        TOI_name = "TIC " + rows[x][fields.index("TIC ID")] 
        period = float(rows[x][fields.index("Period (days)")])
        t0 = float(rows[x][fields.index("Epoch (BJD)")])
        duration_hours = float(rows[x][fields.index("Duration (hours)")])
        fractional_duration = (duration_hours / 24.0) / period

        if(duration_hours < 3.0): 
            
            if(fractional_duration < 0.004): 
                search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = (temp_fold.phase > -300*fractional_duration) & (temp_fold.phase < 300.0*fractional_duration)
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.scatter(ax=axes, color="black")
                fig.savefig(path + "fig" + str(x) + ".pdf")
                plt.close()
            
            if(fractional_duration < 0.01):
                search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = (temp_fold.phase > -15*fractional_duration) & (temp_fold.phase < 15.0*fractional_duration)
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0044) 
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.scatter(ax=axes, color="black")
                fig.savefig(path + "fig" + str(x) + ".pdf")
                plt.close()
            
            elif(fractional_duration < 0.04):
                search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = (temp_fold.phase > -4*fractional_duration) & (temp_fold.phase < 4.0*fractional_duration) 
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.scatter(ax=axes, color="black")
                fig.savefig(path + "fig" + str(x) + ".pdf")
                plt.close()
            
            else:
                search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
                transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])

                lc_flat = lc_clean.flatten(mask=transit_mask)

                lc_fold = lc_flat.fold(period, epoch_time=(t0-2457000))

                lc_global = lc_fold.bin(time_bin_size=0.0008)

                phase_mask = (lc_fold.phase > -15*fractional_duration) & (lc_fold.phase < 15.0*fractional_duration) 
                lc_zoom = lc_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)  
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))

                fig, axes = plt.subplots()
                lc_local.scatter(ax=axes, color="black")
                fig.savefig(path + "fig" + str(x) + ".pdf")
                plt.close()
    
        elif(duration_hours < 5.0):
            search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
            if(len(search_result) == 0):
                search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)

            lcs = search_result.download_all() 
            lc_raw = lcs.stitch() 

            lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

            temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

            fractional_duration = (duration_hours / 24.0) / period

            phase_mask = (temp_fold.phase > -25*fractional_duration) & (temp_fold.phase < 25.0*fractional_duration) 
            lc_zoom = temp_fold[phase_mask]

            lc_local = lc_zoom.bin(time_bin_size=0.0008)
            lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
            
            fig, axes = plt.subplots()
            lc_local.scatter(ax=axes, color="black")
            fig.savefig(path + "fig" + str(x) + ".pdf")
            plt.close()
        
        else:
            if(fractional_duration < 0.02):
                search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = (temp_fold.phase > -300*fractional_duration) & (temp_fold.phase < 300.0*fractional_duration)
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.scatter(ax=axes, color="black")
                fig.savefig(path + "fig" + str(x) + ".pdf")
                plt.close()
                
            else:
                search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
                transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])

                lc_flat = lc_clean.flatten(mask=transit_mask)

                lc_fold = lc_flat.fold(period, epoch_time=(t0-2457000))

                lc_global = lc_fold.bin(time_bin_size=0.0008) 

                phase_mask = (lc_fold.phase > -15*fractional_duration) & (lc_fold.phase < 15.0*fractional_duration) 
                lc_zoom = lc_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)  
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.scatter(ax=axes, color="black")
                fig.savefig(path + "fig" + str(x) + ".pdf")
                plt.close()
    
    except AttributeError:
        print("AttributeError occured with", TOI_name, "item", str(x))
    except ZeroDivisionError:
        print("ZeroDivisionError occured with", TOI_name, "item", str(x))
    except:
        print("An error occured with", TOI_name, "item", str(x))

An error occured with TIC 394137592 item 20


No data found for target "TIC 265612438".
No data found for target "TIC 403287048".
No data found for target "TIC 253917293".
No data found for target "TIC 99493790".
No data found for target "TIC 79395355".
